In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict
import catboost
 

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/JeffSackmann/tennis_MatchChartingProject/master/charting-m-points.csv',encoding= 'unicode_escape',low_memory=False)

In [3]:
df2 = pd.read_csv('https://raw.githubusercontent.com/JeffSackmann/tennis_MatchChartingProject/master/charting-m-matches.csv', quoting = 3, error_bad_lines = False, encoding = 'unicode_escape')

# Select Player of Choice and Tournamets of Choice(Federer Example)

In [ ]:
df4 = df2[df2['Tournament'].str.contains('Wimbledon|Australian Open|US Open|Roland Garros')]
pd.pandas.set_option('display.max_columns', None)
df3 = pd.merge(df, df4, on='match_id')
rf = df3[df3['match_id'].str.contains("Roger_Federer")]
rf_server = rf[(rf['Serving'] == 'RF')]
rf_server



In [ ]:
rf_not = rf[(rf['Serving'] != 'RF')]
rf_not


# Extract Point Ending Shots from Player of Choice when they Serve

In [4]:
def test_f(string, point_winner, surface, main_df):
    
    test = []
    position = ['1','1']
    shot_direction = ['']
    
    if len(string) < 6:
        return main_df
    
    
    string = string.replace('7','')
    string = string.replace('8','')
    string = string.replace('9','')
    string = string.replace('-', '')
    string = string.replace('=', '')
    string = string.replace('+', '')
    string = string.replace('d', '')
    string = string.replace(';', '')
    string = string.replace('*', '')
    string = string.replace('!', '')
    string = string.replace('^', '')
    string = string.replace('@', '')
    string = string.replace('#', '')


    
   


    
    
    
    for i in range(len(string)):
        
        if string[i] in ('*','#','@','n','w','d','x','g','!','e'):
            break
        elif string[i] == '4':
            test.append('out wide')
        elif string[i] == '5':
            test.append('body')
        elif string[i] == '6':
            test.append('down the t')
        
        elif string[i] == 'f':
            test.append('forehand groundstoke')
        elif string[i] == 'b':
            test.append('backhand groundstroke')
        elif string[i] == 'r':
            test.append('forehand slice')
        elif string[i] == 's':
            test.append('backhand slice')
        elif string[i] == 'v':
            test.append('forehand volley')
        elif string[i] == 'z':
            test.append('backhand volley')
        elif string[i] == 'o':
            test.append('forehand overhead')
        elif string[i] == 'p':
            test.append('backhand overhead')
        elif string[i] == 'u':
            test.append('forehand dropshot')
        elif string[i] == 'y':
            test.append('backhand dropshot')
        elif string[i] == 'l':
            test.append('forehand lob')
        elif string[i] == 'm':
            test.append('backhand lob')
        elif string[i] == 'h':
            test.append('forehand half volley')
        elif string[i] == 'i':
            test.append('backhand half volley')
        elif string[i] == 'j':
            test.append('forehand swing volley')
        elif string[i] == 'k':
            test.append('backhand swing volley')
        elif string[i] == 't':
            test.append('tick shot')
        elif string[i] == 'q':
            test.append('unknown shot')
            
        elif string[i] in ('1','2','3'):
            if string[i] == '1':
                shot_direction.append('right')
            elif string[i] == '2':
                shot_direction.append('middle')
            elif string[i] == '3':
                shot_direction.append('left')
            if string[i-1] in ('f','b','r','s','l','m','t','u','y') and string[i-2] == '1':
                position.append('1')
            elif string[i-1] in ('f','b','r','s','l','m','t','u','y') and string[i-2] == '2':
                position.append('2')
            elif string[i-1] in ('f','b','r','s','l','m','t','u','y') and string[i-2] == '3':
                position.append('3')
            elif string[i-1] == '+'  and string[i-2] in ('f','b','r','s','l','m','t','u','y') and string[i-3] == '1':
                position.append('4')
            elif string[i-1] == '+'  and string[i-2] in ('f','b','r','s','l','m','t','u','y') and string[i-3] == '2':
                position.append('5')
            elif string[i-1] == '+'  and string[i-2] in ('f','b','r','s','l','m','t','u','y') and string[i-3] == '3':
                position.append('6')
            elif string[i-1] in ('v','z','h','i','j','k','o','p') and string[i-2] == '1':
                position.append('4')
            elif string[i-1] in ('v','z','h','i','j','k','o','p') and string[i-2] == '2':
                position.append('5')
            elif string[i-1] in ('v','z','h','i','j','k','o','p') and string[i-2] == '3':
                position.append('6')
                
    raw = defaultdict(list)
    shot = test
    position = position
    direction = shot_direction
    rally_length = len(shot)

    
    
    try:
        if len(direction) != len(shot):
            raise Exception(f"Direction: {shot}, {direction}")
        if len(position) != len(shot):
            raise Exception(f"Position: {shot}, {position}")
        
        
        
        #index = len(shot) - 2
        #if index <= 0:
            #index = 2
    
            
        if len(shot) % 2 == 0:
            #checking if even
            shot.pop()
            direction.pop()
            position.pop()
        
        raw['Rally'].append(rally_length) #Rally Count
        raw['Serving'] = ['yes'] #Is the player of interest serving?
        raw['Outgoing Shot'].append(shot[len(shot)-1]) #players outgoing shot
        raw['Serve'].append(shot[0]) # serve type
        raw['My Prev'].append(position[len(position)-1]) #players position
        raw['Second Prev'].append(position[len(position)-3]) #players previous position
        raw['Incoming Shot'].append(shot[len(shot)-2]) #opponents incoming shot
        raw['Incoming Direction'].append(direction[len(direction)-2]) #opponents incoming shot direction
        raw['Opponent Previous'].append(position[len(position)-2]) #opponents  position
        raw['Opponent_2nd'].append(position[len(position)-4]) #opponents previous position
        raw['Outgoing Direction'].append(direction[len(direction)-1]) #players outgoing shot direction
        raw['Prev Shot'].append(shot[len(shot)-3]) #players previous shot
        raw['Point Winner'].append(point_winner) #did the player win the point? 1/0
        raw['Surface'].append(surface) #surface type
        
        


    
    except Exception as e:
        
        print(string, e, shot, direction, position)
        
        return main_df

 

    df1 = pd.DataFrame.from_dict(raw)
    main_df = main_df.append(df1, ignore_index=True)
    

    return main_df

    
    
    
            

In [ ]:

new = new = rf_server[['1st','2nd','isSvrWinner','Surface']]
new.count()


In [ ]:
list_of_rallies = []
columns = ['My Prev', 'Incoming Shot', 'Incoming Direction', 'Opponent Previous', 'Outgoing Direction', 'Outgoing Shot','Point Winner','Rally','Serving']

df_ = pd.DataFrame(columns=columns)

for i, row in new.iterrows():
    
    try:
        if pd.isnull(row['2nd']) == True:
            res = row['1st']
            df_ = test_f(res, row['isSvrWinner'], row['Surface'],df_)
            
        



        else:
            res = row['2nd']
            df_ = test_f(res, row['isSvrWinner'], row['Surface'],df_)
            #list_of_rallies.append((new.iloc[i,1]))
    except Exception as e:
        print(e)
        continue


In [ ]:
df_['Surface'].fillna(value = 'Grass', inplace=True)

df_.count()

# Extract Point Ending Shots from Player of Interest when they are recieving

In [5]:
def test_f_2(string, point_winner, surface, main_df):
    
    test = []
    position = ['1','1']
    shot_direction = ['']
    
    if len(string) < 6:
        return main_df
    
    
    string = string.replace('7','')
    string = string.replace('8','')
    string = string.replace('9','')
    string = string.replace('-', '')
    string = string.replace('=', '')
    string = string.replace('+', '')
    string = string.replace('d', '')
    string = string.replace(';', '')
    string = string.replace('*', '')
    string = string.replace('!', '')
    string = string.replace('^', '')
    string = string.replace('@', '')
    string = string.replace('#', '')


    
   


    
    
    
    for i in range(len(string)):
        
        if string[i] in ('*','#','@','n','w','d','x','g','!','e'):
            break
        elif string[i] == '4':
            test.append('out wide')
        elif string[i] == '5':
            test.append('body')
        elif string[i] == '6':
            test.append('down the t')
        
        elif string[i] == 'f':
            test.append('forehand groundstoke')
        elif string[i] == 'b':
            test.append('backhand groundstroke')
        elif string[i] == 'r':
            test.append('forehand slice')
        elif string[i] == 's':
            test.append('backhand slice')
        elif string[i] == 'v':
            test.append('forehand volley')
        elif string[i] == 'z':
            test.append('backhand volley')
        elif string[i] == 'o':
            test.append('forehand overhead')
        elif string[i] == 'p':
            test.append('backhand overhead')
        elif string[i] == 'u':
            test.append('forehand dropshot')
        elif string[i] == 'y':
            test.append('backhand dropshot')
        elif string[i] == 'l':
            test.append('forehand lob')
        elif string[i] == 'm':
            test.append('backhand lob')
        elif string[i] == 'h':
            test.append('forehand half volley')
        elif string[i] == 'i':
            test.append('backhand half volley')
        elif string[i] == 'j':
            test.append('forehand swing volley')
        elif string[i] == 'k':
            test.append('backhand swing volley')
        elif string[i] == 't':
            test.append('tick shot')
        elif string[i] == 'q':
            test.append('unknown shot')
            
        elif string[i] in ('1','2','3'):
            if string[i] == '1':
                shot_direction.append('right')
            elif string[i] == '2':
                shot_direction.append('middle')
            elif string[i] == '3':
                shot_direction.append('left')
            if string[i-1] in ('f','b','r','s','l','m','t','u','y') and string[i-2] == '1':
                position.append('1')
            elif string[i-1] in ('f','b','r','s','l','m','t','u','y') and string[i-2] == '2':
                position.append('2')
            elif string[i-1] in ('f','b','r','s','l','m','t','u','y') and string[i-2] == '3':
                position.append('3')
            elif string[i-1] == '+'  and string[i-2] in ('f','b','r','s','l','m','t','u','y') and string[i-3] == '1':
                position.append('4')
            elif string[i-1] == '+'  and string[i-2] in ('f','b','r','s','l','m','t','u','y') and string[i-3] == '2':
                position.append('5')
            elif string[i-1] == '+'  and string[i-2] in ('f','b','r','s','l','m','t','u','y') and string[i-3] == '3':
                position.append('6')
            elif string[i-1] in ('v','z','h','i','j','k','o','p') and string[i-2] == '1':
                position.append('4')
            elif string[i-1] in ('v','z','h','i','j','k','o','p') and string[i-2] == '2':
                position.append('5')
            elif string[i-1] in ('v','z','h','i','j','k','o','p') and string[i-2] == '3':
                position.append('6')
                
    raw = defaultdict(list)
    shot = test
    position = position
    direction = shot_direction
    rally_length = len(shot)

    
    
    try:
        if len(direction) != len(shot):
            raise Exception(f"Direction: {shot}, {direction}")
        if len(position) != len(shot):
            raise Exception(f"Position: {shot}, {position}")
        
        
        
        #index = len(shot) - 2
        #if index <= 0:
            #index = 2
    
            
        if len(shot) % 2 != 0:
            #checking if even
            shot.pop()
            direction.pop()
            position.pop()
        
        raw['Rally'].append(rally_length)
        raw['Serving'] = ['No']
        raw['Outgoing Shot'].append(shot[len(shot)-1])
        raw['Serve'].append(shot[0])
        raw['My Prev'].append(position[len(position)-1])
        raw['Second Prev'].append(position[len(position)-3])
        raw['Incoming Shot'].append(shot[len(shot)-2])
        raw['Incoming Direction'].append(direction[len(direction)-2])
        raw['Opponent Previous'].append(position[len(position)-2])
        raw['Opponent_2nd'].append(position[len(position)-4])
        raw['Outgoing Direction'].append(direction[len(direction)-1])
        raw['Prev Shot'].append(shot[len(shot)-3])
        raw['Point Winner'].append(point_winner)
        raw['Surface'].append(surface)
        
        


    
    except Exception as e:
        
        print(string, e, shot, direction, position)
        
        return main_df

 

    df1 = pd.DataFrame.from_dict(raw)
    main_df = main_df.append(df1, ignore_index=True)
    

    return main_df

    
    
    
            

In [ ]:

new2 = new2 = rf_not[['1st','2nd','isSvrWinner','Surface']]
new2.count()


In [ ]:
list_of_rallies = []
columns = ['Serve','My Prev', 'Incoming Shot', 'Incoming Direction', 'Opponent Previous', 'Outgoing Direction', 'Outgoing Shot','Point Winner','Rally','Serving']

df_2 = pd.DataFrame(columns=columns)

for i, row in new2.iterrows():
    
    if row['isSvrWinner'] == 0:
        row['isSvrWinner'] = 1
    elif row['isSvrWinner'] == 1:
        row['isSvrWinner'] = 0
    try:
        if pd.isnull(row['2nd']) == True:
            res = row['1st']
            df_2 = test_f_2(res, row['isSvrWinner'], row['Surface'],df_2)
            
        



        else:
            res = row['2nd']
            df_2 = test_f_2(res, row['isSvrWinner'], row['Surface'],df_2)
            #list_of_rallies.append((new.iloc[i,1]))
    except Exception as e:
        print(e)
        continue


# Combine Both Datasets

In [ ]:
roger = pd.concat([df_, df_2], ignore_index=False)
roger.count()

# Convert to a csv file

In [ ]:
roger.to_csv ('roger.csv', index = False, header=True)


Once you export all the player of interests shot ending points into csv's you can begin the next step, this tutorial will use Federer, Djokovic and Murrays point ending shots

# Combining the datasets and preparing the x & y matricies

In [ ]:
data = pd.read_csv('murray.csv')


In [ ]:
data2 = pd.read_csv('roger.csv')


In [ ]:
data3 = pd.read_csv('novak.csv')


In [ ]:
full = pd.concat([data, data2], ignore_index=False)
full.count()

In [ ]:
full = pd.concat([full, data3], ignore_index=False)
full = full.dropna()
full.count()

In [ ]:
y = full['Point Winner']
full['Rally'] = full['Rally'].astype('int')
full.dropna(inplace = True)
full.count()


In [ ]:
full['My Prev'] = full['My Prev'].astype('object')
full['Opponent Previous'] = full['Opponent Previous'].astype('object')
full['Second Prev'] = full['Second Prev'].astype('object')
full['Opponent_2nd'] = full['Opponent_2nd'].astype('object')


full.dtypes

In [ ]:
X = full.drop(['Point Winner'], axis=1)
X

# Encoding Features

In [ ]:
from category_encoders.target_encoder import TargetEncoder

tree_data=full.copy()
for col in tree_data.select_dtypes(include='O').columns:
    te=TargetEncoder()
    tree_data[col]=te.fit_transform(tree_data[col],y)

# Train-Test Split

In [ ]:
from sklearn.model_selection import train_test_split

X_train1, X_test1, y_train1, y_test1 = train_test_split(tree_data.loc[:, tree_data.columns != 'Point Winner']
, tree_data['Point Winner'], test_size = 0.3, random_state=101)

# Building an optimal decision tree

A full sized tree is first built using binary recursive splitting. Cost complexity pruning is used to produce multiple subtrees for range of values of alpha. The best value for alpha is then selected using cross validation. Finally, the optimal subtree that corresponds to this alpha is then outputted.

In [ ]:
model2 = DecisionTreeClassifier(criterion='entropy', min_samples_leaf=5)
path = model2.cost_complexity_pruning_path(X_train1, y_train1)
alphas = path.ccp_alphas

In [ ]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

model2 = DecisionTreeClassifier(criterion='entropy', min_samples_leaf=5)

search_space = {
    'ccp_alpha': alphas,
}

tree_search = RandomizedSearchCV(model2, search_space, cv = 5 , scoring='neg_log_loss')
tree_search.fit(X_train1, y_train1)
tree2 = tree_search.best_estimator_

print('Best parameters found by grid search:', tree_search.best_params_, '\n')

In [ ]:
#visualize the tree

import graphviz
from sklearn.tree import export_graphviz

dot_data = export_graphviz(tree2, out_file=None , impurity=False, feature_names = predictors,
                           class_names=['0','1'], rounded=True) 
graph = graphviz.Source(dot_data)
graph.render('tree02') # saves tree to a file
graph


In [ ]:
#evaluate tree
tree_pred = tree2.predict(X_test1)
from sklearn import metrics

print(metrics.roc_auc_score(y_test, tree_pred))
print()

# CatBoost

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(full.loc[:, full.columns != 'Point Winner']
, full['Point Winner'], test_size = 0.3, random_state=101)


In [ ]:
X_train
categorical_features_indices = np.where(X_train.dtypes != np.int)[0]
categorical_features_indices

In [ ]:
from catboost import CatBoostClassifier
model = CatBoostClassifier(
    #custom_loss=['AUC', 'Accuracy'],
    iterations = 100,

    loss_function='Logloss'
)

In [ ]:
y_train=y_train.astype('int')
y_test = y_test.astype('int')

In [ ]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from scipy.stats import uniform as sp_randFloat
from scipy.stats import uniform as sp_randInt
from scipy.stats import randint
from scipy.stats import uniform as unif





In [ ]:
parameters = {
    'learning_rate': unif(0.005,0.1), # uniform distribution between 0.005 and 0.1
    #'n_estimators' : randint(10, 1000), # discrete uniform distribution between 100 and 2500
    'depth': randint(2,12)
}

Grid_CBC = RandomizedSearchCV(estimator=model, param_distributions = parameters,
                               cv = 5, n_jobs=-1)
Grid_CBC.fit(X_train, y_train,
            cat_features=categorical_features_indices)

In [ ]:
print('Best parameters found by randomised search:', Grid_CBC.best_params_, '\n')


In [ ]:
from catboost import CatBoostClassifier, Pool, metrics, cv

model = CatBoostClassifier(
    depth = 11,
    learning_rate = 0.03211884497138581,
    n_estimators = 2000,
    custom_loss=[metrics.AUC()],

    loss_function='Logloss'
)
model.fit(X_train, y_train, cat_features=categorical_features_indices,logging_level='Verbose',
    eval_set=(X_test, y_test), plot=True)

In [ ]:
model.get_feature_importance(prettified=True)


# Another Implementation of Gradient Boosting

In [ ]:
y_train1=y_train1.astype('int')
y_test1 = y_test1.astype('int')

In [ ]:
import lightgbm as lgb
from scipy.stats import uniform as unif
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV



model = lgb.LGBMClassifier()

search_space = {
    'learning_rate': unif(0.005,0.1), # uniform distribution between 0.005 and 0.1
    'n_estimators' : randint(100, 2501), # discrete uniform distribution between 100 and 2500
    'num_leaves' : randint(2, 100), # discrete uniform distribution between 2 and 4
    'subsample' : unif(0.5, 1), # uniform distribution between 0.5 and 1
    'max_depth': randint(2,100),
}


lbst = RandomizedSearchCV(model, search_space, n_iter = 64, cv = 5, n_jobs=4, random_state=1)
lbst.fit(X_train, y_train)




In [ ]:
print('Best parameters found by randomised search:', lbst.best_params_, '\n')


In [ ]:
y_pred=lbst.predict(X_test1)
from sklearn import metrics
y_test1=y_test1.astype('int')

print(metrics.roc_auc_score(y_test1, y_pred))
